In [1]:
import torch

from modeling_dream.tokenization_dream import DreamTokenizer

/home/exx/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def select_device():
    if torch.cuda.is_available():
        return "cuda"
    mps_backend = getattr(torch.backends, "mps", None)
    if mps_backend is not None and mps_backend.is_available():
        return "mps"
    return "cpu"
# end

# --- Model Loading ---
model_path = "Dream-org/Dream-v0-Base-7B"
device = select_device()
dtype_by_device = {
    "cuda": torch.bfloat16,
    "mps": torch.float16,
    "cpu": torch.float32,
}
dtype = dtype_by_device[device]
print(f"Using device: {device} (dtype={dtype})")


Using device: cuda (dtype=torch.bfloat16)


In [ ]:
tokenizer = DreamTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    padding_side='left'
)

In [8]:
for k, v in tokenizer.special_tokens_map.items():
    print(f'{v}: {tokenizer.encode(v)}')
# end

<|beginoftext|>: [151665]
<|endoftext|>: [151643]
<|endoftext|>: [151643]
<|mask|>: [151666]
['<|beginoftext|>', '<|mask|>']: [151665, 151666]


[151666]

In [ ]:

inputs = tokenizer.apply_chat_template(
    messages, return_tensors="pt", return_dict=True, add_generation_prompt=True, padding=True
)
input_ids = inputs.input_ids.to(device)
attention_mask = inputs.attention_mask.to(device)

output = model.diffusion_generate(
